In [ ]:
# Circle Finder CNN - Colab Compatible

In [ ]:
## 1. Install Required Packages
!pip install shapely scikit-image

In [ ]:
## 2. Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
## 3. Dataset Class
class CircleDataset(Dataset):
    def __init__(self, data_path):
        self.files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]
        self.transform = transforms.ToTensor()

In [ ]:
def __len__(self):
        return len(self.files)

In [ ]:
def __getitem__(self, idx):
        with open(self.files[idx], 'r') as f:
            data = json.load(f)
        img = np.array(data['img'], dtype=np.float32)
        label = data['label']
        img = np.expand_dims(img, axis=0)  # Add channel dimension
        target = torch.tensor([label['row'], label['col'], label['radius']], dtype=torch.float32)
        return torch.tensor(img, dtype=torch.float32), target

In [ ]:
## 4. CNN Model
class CircleFinderCNN(nn.Module):
    def __init__(self):
        super(CircleFinderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 3)  # Output: row, col, radius

In [ ]:
def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
## 5. Training Function
def train_model(model, train_loader, val_loader, device, epochs=10, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.to(device)

In [ ]:
for epoch in range(epochs):
        model.train()
        train_loss = 0
        for imgs, targets in train_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

In [ ]:
val_loss = 0
        model.eval()
        with torch.no_grad():
            for imgs, targets in val_loader:
                imgs, targets = imgs.to(device), targets.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

In [ ]:
print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

In [ ]:
## 6. Load Data and Train
data_dir = './data'  # Ensure this folder contains your 1000 .json files
full_dataset = CircleDataset(data_dir)
train_idx, val_idx = train_test_split(list(range(len(full_dataset))), test_size=0.2)
train_dataset = torch.utils.data.Subset(full_dataset, train_idx)
val_dataset = torch.utils.data.Subset(full_dataset, val_idx)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CircleFinderCNN()
train_model(model, train_loader, val_loader, device, epochs=10)

In [ ]:
## 7. Evaluate with IoU

In [ ]:
from shapely.geometry import Point

In [ ]:
def intersection_over_union(circ1_dict, circ2_dict):
    shape1 = Point(circ1_dict['row'], circ1_dict['col']).buffer(circ1_dict['radius'])
    shape2 = Point(circ2_dict['row'], circ2_dict['col']).buffer(circ2_dict['radius'])
    return shape1.intersection(shape2).area / shape1.union(shape2).area

In [ ]:
def evaluate_model(model, dataloader, device, iou_threshold=0.7):
    model.eval()
    matches = 0
    total = 0
    with torch.no_grad():
        for imgs, targets in dataloader:
            imgs = imgs.to(device)
            outputs = model(imgs).cpu().numpy()
            targets = targets.numpy()
            for pred, true in zip(outputs, targets):
                pred_dict = {'row': pred[0], 'col': pred[1], 'radius': pred[2]}
                true_dict = {'row': true[0], 'col': true[1], 'radius': true[2]}
                iou = intersection_over_union(pred_dict, true_dict)
                if iou > iou_threshold:
                    matches += 1
                total += 1
    accuracy = matches / total
    print(f"IoU > {iou_threshold} Accuracy: {accuracy:.2%}")
    return accuracy

In [ ]:
# Run evaluation
evaluate_model(model, val_loader, device)

In [ ]:
## 8. Visualize Predictions

In [ ]:
def show_predictions(model, dataset, device, num_samples=5):
    model.eval()
    indices = np.random.choice(len(dataset), num_samples, replace=False)
    fig, axs = plt.subplots(1, num_samples, figsize=(15, 3))
    for i, idx in enumerate(indices):
        img, true = dataset[idx]
        with torch.no_grad():
            pred = model(img.unsqueeze(0).to(device)).cpu().numpy()[0]
        axs[i].imshow(img.squeeze(), cmap='gray')
        axs[i].set_title(f"True: ({int(true[0])}, {int(true[1])}, {int(true[2])})\nPred: ({int(pred[0])}, {int(pred[1])}, {int(pred[2])})")
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Show some predictions
show_predictions(model, val_dataset, device, num_samples=5)